## 3. Analisador de Padrões de Vento e Rosa dos Ventos Avançada
### Construa um sistema completo de análise de ventos:

- Processe dados de direção e velocidade do vento em múltiplas estações
- Calcule estatísticas direcionais, distribuições de Weibull e potencial eólico
- Gere rosas dos ventos interativas, mapas de turbulência e análises de cisalhamento vertical
- Produza relatórios técnicos para avaliação de sites eólicos com análise de viabilidade energética

## Dependencias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.projections import PolarAxes
import mpl_toolkits.axisartist.grid_finder as gf
import mpl_toolkits.axisartist.floating_axes as fa
from scipy.stats import weibull_min
from typing import Dict, List, Optional, Tuple

## Desenvolvimento do Código

In [ ]:
class AnalisadorVento:
    """
    Classe para análise avançada de padrões de vento e geração de rosas dos ventos.
    
    Atributos:
        dados (pd.DataFrame): DataFrame contendo os dados de vento
        estacoes (List[str]): Lista de estações disponíveis nos dados
        parametros_weibull (Dict): Dicionário com parâmetros de Weibull calculados
    """

    def __init__(self, dados: pd.DataFrame):
        """
        Inicializa o analisador com os dados de vento.
        
        Args:
            dados (pd.DataFrame): DataFrame com colunas:
                - 'estacao': nome da estação
                - 'data': timestamp da medição
                - 'direcao': direção do vento em graus (0-360)
                - 'velocidade': velocidade do vento em m/s
        """
        self.dados = dados.copy()
        self.estacoes = self.dados['estacao'].unique().tolist()
        self.parametros_weibull = ()

        # pre processamento
        self._preprocessar_dados()
    
    def _preprocessar_dados(self) -> None:
        """Realiza pré-processamento dos dados (limpeza, normalização)."""
        # Remover dados nulos
        self.dados.dropna(subset=['direcao', 'velocidade'], inplace=True)
        # Garantir que direção está entre 0-360
        self.dados['direcao'] = self.dados['direcao'] % 360
        # Filtrar velocidades negativas
        self.dados = self.dados[self.dados['velocidade'] >= 0]
    
    def _calcular_estatisticas(self, estacao: str = None) -> Dict:
        """
        Calcula estatísticas descritivas para os dados de vento.
        
        Args:
            estacao (str): Nome da estação para filtrar (None para todas)
            
        Returns:
            Dict: Dicionário com estatísticas calculadas
        """

        dados = self.dados if estacao is None else self.dados[self.dados['estacao'] == 'estacao']

        estatisticas = {
            'media_velocidade': dados['velocidade'].mean(),
            'max_velocidade': dados['velocidade'].max(),
            'min_velocidade': dados['velocidade'].min(),
            'desvio_velocidade': dados['velocidade'].std(),
            'media_direcao': self._calcular_media_direcao(dados['direcao']),
            'frequencia_calmar': (dados['velocidade'] < 0.5).mean(),
            'turbulencia': dados['velocidade'].std() / dados['velocidade'].mean()
        }

        return estatisticas
    
    def _calcular_direcao_media(self, direcoes: pd.Series) -> float:
        """
        Calcula a direção media de um conjunto de direções.
        
        Args:
            direcoes (pd.Series): Série de direções, em graus
            
        Returns:
            float: Direção media
        """
        radianos = np.deg2rad(direcoes)
        media_sen = np.sin(radianos).mean()
        media_cos = np.cos(radianos).mean()
        media_rad = np.arctan2(media_sen, media_cos)
        return (np.rad2deg(media_rad) + 360) % 360

    def ajustar_distribuicao_weibull(self, estacao: str = None, setores: int = 16) -> Dict:
        """
        Ajusta distribuição de Weibull para os dados de velocidade do vento.
        
        Args:
            estacao (str): Nome da estação para filtrar (None para todas)
            setores (int): Número de setores direcionais (default: 16)
            
        Returns:
            Dict: Parâmetros de Weibull (k, c) por setor direcional
        """
        dados = self.dados if estacao is None else self.dados[self.dados['estacao'] == estacao]

        # Calculo dos limites dos setores
        limites_setores = np.linspace(0,360, setores + 1)

        parametros = ()
        for i in range(setores):
            limite_inf = limites_setores[i]
            limite_sup = limites_setores[i+1]

            # Filtrar dados do setor
            if i == setores - 1:
                mascara = (dados['direcao'] >= limite_inf) & (dados['direcao'] <= limite_sup)
            else:
                mascara = (dados['direcao'] >= limite_inf) & (dados['direcao'] < limite_sup)

            dados_setor = dados[mascara]['velocidade']

            # Ajustar Weibull se houver dados suficientes
            if len(dados_setor) > 10:
                shape, loc, scale = weibull_min.fit(dados_setor, floc=0)
                parametros[f'setor_{i}'] =  {'k': shape, 'c': scale, 'frequencia': len(dados_setor)/len(dados)}
        
        # Armazenar parametros para uso posterior
        chave = estacao if estacao else 'global'
        self.parametros_weibull[chave] = parametros

        return parametros